In [112]:
import pandas as pd
import requests
df_agri=pd.read_csv("./Cleaned CSVs/Banana.csv")
df_agri['Price Date'] = pd.to_datetime(df_agri['Price Date'])
df_agri.head()


,District Name,Market Name,Commodity,Variety,Grade,Min Price (Rs./Quintal),Max Price (Rs./Quintal),Modal Price (Rs./Quintal),Price Date,Day Of Week
0,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4000.0,4500.0,4500.0,2025-06-18,2
1,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4000.0,4500.0,4500.0,2024-12-26,3
2,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2025-06-01,6
3,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2025-06-17,1
4,Dharmapuri,AJattihalli(Farmers Market),Banana,Besrai,Local,4500.0,5000.0,5000.0,2024-11-18,0


In [113]:
CROP_LOOKBACK_DAYS = {
    'Bajra(Pearl Millet/Cumbu)': 90,
    'Maize': 120,
    'Paddy': 135,
    'Banana' : 360,
    'Banana - Green': 180,
    'Black Gram Dal (Urd Dal)': 90
}

In [114]:
DISTRICT_COORDINATES = {
    "Thiruvannamalai": {
        "lat": 9.539867,
        "lon": 77.6197904
    },
    "Kancheepuram": {
        "lat": 12.8796051,
        "lon": 79.7042762
    },
    "Namakkal": {
        "lat": 11.3033889,
        "lon": 78.118604
    },
    "Villupuram": {
        "lat": 11.9398285,
        "lon": 79.4945645
    },
    "Ariyalur": {
        "lat": 11.1530559,
        "lon": 79.2585797
    },
    "Cuddalore": {
        "lat": 11.520207,
        "lon": 79.3396356
    },
    "Coimbatore": {
        "lat": 11.0018115,
        "lon": 76.9628425
    },
    "Salem": {
        "lat": 11.6469616,
        "lon": 78.2106958
    },
    "Vellore": {
        "lat": 12.9021849,
        "lon": 79.061104
    },
    "Erode": {
        "lat": 11.4905281,
        "lon": 77.3505274
    },
    "Theni": {
        "lat": 9.8692558,
        "lon": 77.4222974
    },
    "Thiruchirappalli": {
        "lat": 10.7905,
        "lon": 78.7047
    },
    "Thiruvellore": {
        "lat": 13.1227,
        "lon": 79.9118
    },
    "Nagercoil (Kannyiakumari)": {
        "lat": 8.1833,
        "lon": 77.4119
    },
    "Thirunelveli": {
        "lat": 8.7150,
        "lon": 77.7656
    },
    "Virudhunagar": {
        "lat": 9.4926289,
        "lon": 77.8631831
    },
    "Dindigul": {
        "lat": 10.4256896,
        "lon": 77.815664
    },
    "Ramanathapuram": {
        "lat": 9.5205576,
        "lon": 78.5184635
    },
    "Tuticorin": {
        "lat": 8.8052602,
        "lon": 78.1452745
    },
    "Madurai": {
        "lat": 9.9261153,
        "lon": 78.1140983
    },
    "Karur": {
        "lat": 10.8217671,
        "lon": 78.3828654
    },
    "Dharmapuri": {
        "lat": 12.145613,
        "lon": 78.1132217
    },
    "Pudukkottai": {
        "lat": 10.2903072,
        "lon": 78.8173617
    },
    "Ranipet": {
        "lat": 12.9186243,
        "lon": 79.4081921
    },
    "Krishnagiri": {
        "lat": 12.5152075,
        "lon": 78.0093769
    },
    "The Nilgiris": {
        "lat": 11.4,
        "lon": 76.7
    },
    "Sivaganga": {
        "lat": 9.8486881,
        "lon": 78.4870461
    },
    "Chengalpattu": {
        "lat": 12.6840886,
        "lon": 79.9836371
    },
    "Thirupur": {
        "lat": 12.921745,
        "lon": 80.1375926
    },
    "Kallakuruchi": {
        "lat": 11.4248929,
        "lon": 78.2652132
    },
    "Thanjavur": {
        "lat": 10.659037,
        "lon": 79.2014278
    },
    "Nagapattinam": {
        "lat": 10.6026202,
        "lon": 79.7619348
    },
    "Thiruvarur": {
        "lat": 10.6878621,
        "lon": 79.4979437
    },
    "Thirupathur": {
        "lat": 10.59635,
        "lon": 79.56857
    },
    "Perambalur": {
        "lat": 11.2902641,
        "lon": 78.9300404
    },
    "Tenkasi": {
        "lat": 9.0933908,
        "lon": 77.4758373
    }
}

In [115]:
districts = {k: v for k, v in DISTRICT_COORDINATES.items() if k in df_agri['District Name'].unique()}
lookbacks = {k: v for k, v in CROP_LOOKBACK_DAYS.items() if k in df_agri['Commodity'].unique()}

print(f"Districts being processed for this file: {list(districts.keys())}")
print(f"Lookbacks being processed for this file: {lookbacks}")

Districts being processed for this file: ['Thiruvannamalai', 'Kancheepuram', 'Namakkal', 'Villupuram', 'Ariyalur', 'Cuddalore', 'Coimbatore', 'Salem', 'Vellore', 'Erode', 'Theni', 'Thiruchirappalli', 'Thiruvellore', 'Nagercoil (Kannyiakumari)', 'Thirunelveli', 'Virudhunagar', 'Dindigul', 'Ramanathapuram', 'Tuticorin', 'Madurai', 'Karur', 'Dharmapuri', 'Pudukkottai', 'Ranipet', 'Krishnagiri', 'The Nilgiris', 'Sivaganga', 'Chengalpattu', 'Thirupur', 'Kallakuruchi', 'Thanjavur', 'Nagapattinam', 'Thiruvarur', 'Thirupathur', 'Perambalur', 'Tenkasi']
Lookbacks being processed for this file: {'Banana': 360}


In [116]:
all_weather_data = []
unique_lookbacks = set(lookbacks.values())
start_date = df_agri['Price Date'].min() - pd.Timedelta(days=max(unique_lookbacks, default=0))
end_date = df_agri['Price Date'].max()

print(unique_lookbacks)

{360}


In [117]:
for district, coords in districts.items():
    print(f"Fetching weather data for {df_agri['Commodity'].unique()} in {district}...")
    
    weather_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={coords['lat']}&longitude={coords['lon']}&start_date={start_date.strftime('%Y-%m-%d')}&end_date={end_date.strftime('%Y-%m-%d')}&daily=temperature_2m_mean,precipitation_sum"
    response = requests.get(weather_url)
    
    if response.status_code == 200:
        df_weather = pd.DataFrame(response.json()['daily'])
        df_weather['Price Date'] = pd.to_datetime(df_weather['time'])
        df_weather['District Name'] = district

        for days in unique_lookbacks:
            df_weather[f'temp_mean_{days}d'] = df_weather['temperature_2m_mean'].rolling(window=days, min_periods=1).mean()
            df_weather[f'precip_sum_{days}d'] = df_weather['precipitation_sum'].rolling(window=days, min_periods=1).sum()
        
        all_weather_data.append(df_weather)

df_weather_master = pd.concat(all_weather_data)

Fetching weather data for ['Banana'] in Thiruvannamalai...
Fetching weather data for ['Banana'] in Kancheepuram...
Fetching weather data for ['Banana'] in Namakkal...
Fetching weather data for ['Banana'] in Villupuram...
Fetching weather data for ['Banana'] in Ariyalur...
Fetching weather data for ['Banana'] in Cuddalore...
Fetching weather data for ['Banana'] in Coimbatore...
Fetching weather data for ['Banana'] in Salem...
Fetching weather data for ['Banana'] in Vellore...
Fetching weather data for ['Banana'] in Erode...
Fetching weather data for ['Banana'] in Theni...
Fetching weather data for ['Banana'] in Thiruchirappalli...
Fetching weather data for ['Banana'] in Thiruvellore...
Fetching weather data for ['Banana'] in Nagercoil (Kannyiakumari)...
Fetching weather data for ['Banana'] in Thirunelveli...
Fetching weather data for ['Banana'] in Virudhunagar...
Fetching weather data for ['Banana'] in Dindigul...
Fetching weather data for ['Banana'] in Ramanathapuram...
Fetching weathe

ValueError: No objects to concatenate

In [100]:
print(df_weather_master)

            time  temperature_2m_mean  precipitation_sum Price Date  \
0     2021-10-05                 26.6              103.0 2021-10-05   
1     2021-10-06                 25.8                4.3 2021-10-06   
2     2021-10-07                 27.2                0.1 2021-10-07   
3     2021-10-08                 27.3                1.6 2021-10-08   
4     2021-10-09                 26.8                0.3 2021-10-09   
...          ...                  ...                ...        ...   
1381  2025-07-17                 30.1                0.0 2025-07-17   
1382  2025-07-18                 29.8                0.0 2025-07-18   
1383  2025-07-19                 29.1                0.1 2025-07-19   
1384  2025-07-20                 28.8                0.5 2025-07-20   
1385  2025-07-21                 27.8                0.3 2025-07-21   

        District Name  temp_mean_90d  precip_sum_90d  
0     Thiruvannamalai      26.600000           103.0  
1     Thiruvannamalai      26.200000 

In [101]:
df_merged = pd.merge(df_agri, df_weather_master, on=['District Name', 'Price Date'], how='left')
print(df_merged)

     District Name      Market Name                  Commodity Variety  Grade  \
0         Ariyalur        Andimadom  Bajra(Pearl Millet/Cumbu)   Other  Local   
1         Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
2         Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
3         Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
4        Cuddalore        Cuddalore  Bajra(Pearl Millet/Cumbu)    Bold    FAQ   
...            ...              ...                        ...     ...    ...   
1549    Villupuram       Villupuram  Bajra(Pearl Millet/Cumbu)   Other  Local   
1550    Villupuram       Villupuram  Bajra(Pearl Millet/Cumbu)   Other  Local   
1551     Cuddalore    Virudhachalam  Bajra(Pearl Millet/Cumbu)   Other  Local   
1552     Cuddalore    Virudhachalam  Bajra(Pearl Millet/Cumbu)   Other  Local   
1553     Cuddalore    Virudhachalam  Bajra(Pearl Millet/Cumbu)   Other  Local   

      Min Price (Rs./Quinta

In [102]:
def get_correct_lookback_value(row, feature_prefix, lookback_dict):
    commodity = row['Commodity']
    # ** NEW: Check if the commodity exists in our lookback dictionary **
    if commodity in lookback_dict:
        lookback_days = lookback_dict[commodity]
        column_name = f"{feature_prefix}_{lookback_days}d"
        return row[column_name]
    return None

df_merged['lookback_temp_mean'] = df_merged.apply(get_correct_lookback_value, feature_prefix='temp_mean', lookback_dict=lookbacks, axis=1)
df_merged['lookback_precip_sum'] = df_merged.apply(get_correct_lookback_value, feature_prefix='precip_sum', lookback_dict=lookbacks, axis=1)


In [103]:
print(df_merged.head())

  District Name      Market Name                  Commodity Variety  Grade  \
0      Ariyalur        Andimadom  Bajra(Pearl Millet/Cumbu)   Other  Local   
1      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
2      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
3      Ariyalur  Ariyalur Market  Bajra(Pearl Millet/Cumbu)   Other    FAQ   
4     Cuddalore        Cuddalore  Bajra(Pearl Millet/Cumbu)    Bold    FAQ   

   Min Price (Rs./Quintal)  Max Price (Rs./Quintal)  \
0                   2500.0                   2575.0   
1                   2031.0                   2400.0   
2                   2223.0                   4231.0   
3                   2417.0                   2551.0   
4                   2175.0                   2175.0   

   Modal Price (Rs./Quintal) Price Date  Day Of Week        time  \
0                     2550.0 2025-04-09            2  2025-04-09   
1                     2350.0 2022-09-27            1  2022-09-27

In [104]:
final_columns = [
    'District Name', 'Market Name', 'Commodity', 'Variety', 'Grade', 'Min Price (Rs./Quintal)', 
    'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)', 'Price Date', 'Day Of Week',
    'lookback_temp_mean', 'lookback_precip_sum'
]

df_final = df_merged[[col for col in final_columns if col in df_merged.columns]].copy()


In [105]:
df_final.to_csv('./Weather_Merged_CSVs/Banana.csv', index=False)